With thanks to Ben Leighton, adapted from CCHaPS hindcast rechunking code

In [ ]:
from dask_jobqueue import PBSCluster
from dask.distributed import Client
cluster = PBSCluster(cores=1, walltime="2:00:00", memory="128GB", processes=1,
                     queue='normal', resource_spec="ncpus=1,mem=128GB,jobfs=100GB",
                     python="python3")
cluster.scale(jobs=40)
cluster

In [ ]:
client = Client(cluster)

In [1]:
import xarray as xr
import glob
import shutil
import pandas as pd
import shutil
from rechunker import rechunk
import numpy as np
import zarr
import dask
import s3fs

In [2]:
zarr_path = "s3://gbr-dms-data-public/aims-ereefs-biogeochem-baseline-daily-monthly/data.zarr/"

fs = s3fs.S3FileSystem(anon=True)
data = xr.open_zarr(s3fs.S3Map(zarr_path, s3=fs), consolidated=True)

# Selecting the depth larger than 13, from -5.55m to surface
# data = data.sel(k=slice(target_depth_min, target_depth_max))

# data = data[variables]
data

<xarray.Dataset> Size: 4TB
Dimensions:          (time: 3073, k: 17, latitude: 723, longitude: 491)
Coordinates:
  * latitude         (latitude) float64 6kB -28.7 -28.67 ... -7.066 -7.036
  * longitude        (longitude) float64 4kB 142.2 142.2 142.2 ... 156.8 156.9
  * time             (time) datetime64[ns] 25kB 2010-12-01T02:00:00 ... 2019-...
    zc               (k) float64 136B dask.array<chunksize=(17,), meta=np.ndarray>
Dimensions without coordinates: k
Data variables: (12/108)
    BOD              (time, k, latitude, longitude) float32 74GB dask.array<chunksize=(1, 17, 723, 491), meta=np.ndarray>
    CH_N             (time, latitude, longitude) float32 4GB dask.array<chunksize=(1, 723, 491), meta=np.ndarray>
    CO32             (time, k, latitude, longitude) float32 74GB dask.array<chunksize=(1, 17, 723, 491), meta=np.ndarray>
    CS_Chl           (time, latitude, longitude) float32 4GB dask.array<chunksize=(1, 723, 491), meta=np.ndarray>
    CS_N             (time, latitude, longitude) float32 4GB dask.array<chunksize=(1, 723, 491), meta=np.ndarray>
    CS_bleach        (time, latitude, longitude) float32 4GB dask.array<chunksize=(1, 723, 491), meta=np.ndarray>
    ...               ...
    eta              (time, latitude, longitude) float32 4GB dask.array<chunksize=(1, 723, 491), meta=np.ndarray>
    month_EpiPAR_sg  (time, latitude, longitude) float32 4GB dask.array<chunksize=(1, 723, 491), meta=np.ndarray>
    omega_ar         (time, k, latitude, longitude) float32 74GB dask.array<chunksize=(1, 17, 723, 491), meta=np.ndarray>
    pco2surf         (time, k, latitude, longitude) float32 74GB dask.array<chunksize=(1, 17, 723, 491), meta=np.ndarray>
    salt             (time, k, latitude, longitude) float32 74GB dask.array<chunksize=(1, 17, 723, 491), meta=np.ndarray>
    temp             (time, k, latitude, longitude) float32 74GB dask.array<chunksize=(1, 17, 723, 491), meta=np.ndarray>
Attributes: (12/19)
    Conventions:                   CF-1.0
    NCO:                           netCDF Operators version 4.7.7 (Homepage =...
    RunID:                         2
    _CoordSysBuilder:              ucar.nc2.dataset.conv.CF1Convention
    aims_ncaggregate_buildDate:    2021-09-07T21:05:18+10:00
    aims_ncaggregate_datasetId:    products__ncaggregate__ereefs__GBR4_H2p0_B...
    ...                            ...
    metadata_link:                 https://eatlas.org.au/data/uuid/61f3a6df-2...
    paramfile:                     /home/bai155/EMS_solar2/gbr4_H2p0_B3p1_Cb/...
    paramhead:                     eReefs 4 km grid. SOURCE Catchments with 2...
    technical_guide_link:          https://eatlas.org.au/pydio/public/aims-er...
    technical_guide_publish_date:  2020-08-18
    title:                         eReefs AIMS-CSIRO GBR4 BioGeoChemical 3.1 ...

In [ ]:
tmp_store_path = '/scratch/nf33/bxh595/tmp/test_tmp.zarr'
target_store_path = '/scratch/nf33/bxh595/aims-ereefs-biogeochem-baseline-daily-monthly-rechunk/bgc_all.zarr'

In [ ]:
target_chunks = {"time": -1, "k": 1, "longitude": 82, "latitude": 104} 
max_mem = "40GB"
array_plan = rechunk(
    data,
    target_chunks,
    max_mem,
    target_store_path,
    target_options={
        'overwrite': True,
        #'consolidate_metadata': True  # Consolidate metadata when writing to target_store
    },
    temp_store=tmp_store_path,
    temp_options={
        'overwrite': True,
        #'consolidate_metadata': True  # Optionally consolidate metadata in the temp store as well
    }
)
zarr.consolidate_metadata(target_store_path)

In [ ]:
result = array_plan.execute()

In [ ]:
cluster.scale(0)